# Re-implementing procedure outlined in "Entity Profiling in Knowledge Graphs" (Zhang Et al.)
# This notebook will implement the candidate label creation step
Using a subset of wikidata related to Q44 ("beer")

In [68]:
import os

In [69]:
work_dir = "/Users/nicklein/Documents/grad_school/Research/data"

In [70]:
os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(work_dir)
os.environ['DATA'] = "{}/Q44".format(work_dir)
os.environ['NAME'] = "Q44"
os.environ["OUT"] = "{}/Q44_profiler_output".format(work_dir)

Helper function for executing commands

In [71]:
def run_command(command, substitution_dictionary = {}):
    """Run a templetized command."""
    #cmd = command.replace("NAME", subset_name)
    for k, v in substitution_dictionary.items():
        cmd = cmd.replace(k, v)
    
    print(cmd)
    output = subprocess.run([cmd], shell=True, universal_newlines=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(output.stdout)
    print(output.stderr)

## Examples of using query command:
### Ex - Counting instances of <type, property, value> along with the property label

In [131]:
!kgtk query -i $DATA/$NAME.part.wikibase-item.tsv -i $DATA/$NAME.label.en.tsv --graph-cache $STORE \
--match 'item: (n1)-[l {label: p}]->(n2), item: (n1)-[:P31]->(type), label: (p)-[:label]->(lab)' \
--return 'distinct type as type, l.label as prop, n2 as value, count(n1) as count, lab as property_label' \
--where 'lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'type, count(n1) desc' \
--limit 5

type	prop	value	count	property_label
Q1066984	P1343	Q97879676	3	'described by source'@en
Q1066984	P1343	Q316838	2	'described by source'@en
Q1066984	P17	Q154195	2	'country'@en
Q1066984	P17	Q183	2	'country'@en
Q1066984	P17	Q2415901	2	'country'@en


### Ex - Counting property, value pairs for time properties

In [165]:
!kgtk query -i $DATA/$NAME.part.time.tsv -i $DATA/$NAME.label.en.tsv --graph-cache $STORE \
--match 'time: (n1)-[l {label: p}]->(n2), label: (p)-[:label]->(lab)' \
--return 'distinct l.label as prop, lab as property_label, kgtk_date_year(n2) as val, count(n1) as count' \
--where 'lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'count(n1) desc' \
--limit 5

prop	property_label	val	count
P571	'inception'@en	1991	20
P571	'inception'@en	1918	14
P571	'inception'@en	1989	8
P571	'inception'@en	1996	8
P571	'inception'@en	1821	6


### Ex- Counting property, value pairs for quantity properties

In [133]:
!kgtk query -i $DATA/$NAME.part.quantity.tsv -i $DATA/$NAME.label.en.tsv --graph-cache $STORE \
--match 'quantity: (n1)-[l {label: p}]->(n2), label: (p)-[:label]->(lab)' \
--return 'distinct l.label as prop, lab as property_label, n2 as value, count(n1) as count' \
--where 'lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'count(n1) desc' \
--limit 5

prop	property_label	value	count
P3000	'marriageable age'@en	+18Q24564698	56
P2997	'age of majority'@en	+18Q24564698	52
P1279	'inflation rate'@en	+1.7Q11229	41
P2884	'mains voltage'@en	+230Q25250	40
P1279	'inflation rate'@en	+1.8Q11229	38


# Outline of procedure:
**Goal**:<br>
We want to create candidate label sets including
- Attribute value labels (type, property, *attribute*)
- Realtional entity labels (type, property, *entity*)
- Attribute interval labels (type, property, *range of attribute values*)
- Relational attribute labels (type, property, *attribute or attribute range of another entity*)

To enable subsequent filtering of these labels, we also want to count:
- The number of entities of each type
- The number of entities that match each label (call these "positives")
    
**Steps**:

0. Create type-mapping
1. Count the number of entities of each type
    - *optional future step*: define type with P279 transitive closure in addition to P31. 
2. Create AVLs trivially from attribute files along with counts of the positive entities for each label
    - **TODO:** create labels for all types that we care about. Currently done string and time. Should add quantities as well
    - At this step, we should also contribute to a mapping of entities --> matching attribute labels to facilitate creating RALs in a later step  
3. Create RELs trivially from entity relation files along with counts of positive entities for each label
4. Create AILs by discretizing the AVLs we found, along with counts of positive entities for each label
    - This can be done using the counts that we already have
    - We'll likely try a naive approach at first such as hardcoding a bucket size for each numeric type we care about
    - At this step, we should also contribute to a mapping of entities --> matching attribute labels to facilitate creating RALs in a later step
    - **kind-of Issue:** in naive bucketing method, run into mod operator doesn't seem to be supported in query
    - **TODO:** make table of type/property --> low_bound/up_bound buckets to be used instead of using mod operator.
    - to do the above it would be nice to have some -/+ infinity value. Otherwise, need logic like "WHERE low_bound is null OR low_bound < value" - this might work?
5. Create RALs by using the REL table and the entities --> attribute labels table that built in steps 2 and 4. Also keep track of counts of positive entities for each label
6. Filter the labels using a simple heuristic based on counts
    - **TODO**
    
*Misc issues encountered*
- Finding that some of the kgtk type interpretation functions aren't recognized (e.g. I can use kgtk_date_year no problem, but kgtk_quantity_number_int not recognized)
- Is there a way to make an id by concatenating multiple values in the row? I found stringify, but not sure how to concatenate strings.
    - could be useful for giving labels ids and then we could look up what labels an entity had by their id rather than type prop and value
    - --> yes. kgtk add-id
- kgtk rename-columns doesn't always work when input file == output file. Getting around this right now by creating extra temp files... 
- mod operator not supported in a query?
- is there a notion of infinity/-infinity that I can put in a kgtk file?

## 0. Create type mapping
Using P31 only for now, but can add P279* as well later

In [361]:
!kgtk filter -p ' ; P31 ; ' -i $DATA/$NAME.part.wikibase-item.tsv -o $OUT/$NAME.type_mapping.tsv

In [364]:
!head -5 $OUT/$NAME.type_mapping.tsv

id	node1	label	node2
Q1000597-P31-1	Q1000597	P31	Q3957
Q1011-P31-2	Q1011	P31	Q112099
Q1011-P31-1	Q1011	P31	Q3624078
Q1019-P31-2	Q1019	P31	Q112099


## 1. Count number of entities of each type:

In [365]:
!kgtk query -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.label.en.tsv -o $OUT/$NAME.entity_counts_per_type.tsv --graph-cache $STORE \
--match 'type: (n1)-[]->(type), label: (type)-[:label]->(lab)' \
--return 'distinct type as node1_x, count(distinct n1) as node2_x, lab as entity_label, "_" as id' \
--where 'lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'count(distinct n1) desc' \

In [366]:
!kgtk rename-columns -i $OUT/$NAME.entity_counts_per_type.tsv -o $OUT/$NAME.entity_counts_per_type.tsv \
--old-columns node1_x node2_x --new-columns node1 node2 

In [367]:
!head -5 $OUT/$NAME.entity_counts_per_type.tsv

node1	node2	entity_label	id
Q131734	87	'brewery'@en	_
Q3624078	69	'sovereign state'@en	_
Q4830453	50	'business'@en	_
Q6256	26	'country'@en	_


## 2. Create AVLs with counts of positive entities
### Keep track of entities --> matching attribute labels for future use
### Keep numeric valued labels and non-numeric valued labels separate to help with discretization at a later step


### todo - use a loop to do this for all types we care about (e.g. day / month / year) currently just doing this for years

### strings
Creating mapping of entity --> string attribute labels

In [437]:
!kgtk query -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.part.string.tsv -i $DATA/$NAME.label.en.tsv \
-o $OUT/$NAME.entity_attribute_labels_string_temp.tsv --graph-cache $STORE \
--match 'string: (n1)-[l {label:p}]->(n2), type: (n1)-[]->(type), label: (p)-[:label]->(lab)' \
--return 'n1 as entity, type as type, p as prop, n2 as value, lab as property_label, "_" as id' \
--where 'lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'n1'

In [438]:
!kgtk rename-columns -i $OUT/$NAME.entity_attribute_labels_string_temp.tsv \
-o $OUT/$NAME.entity_attribute_labels_string_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [439]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.entity_attribute_labels_string_temp1.tsv \
-o $OUT/$NAME.entity_attribute_labels_string_temp2.tsv

In [440]:
!kgtk add-id -i $OUT/$NAME.entity_attribute_labels_string_temp2.tsv \
-o $OUT/$NAME.entity_attribute_labels_string.tsv

In [441]:
!head -5 $OUT/$NAME.entity_attribute_labels_string.tsv

entity	node1	label	node2	property_label	id
Q1000597	Q3957	P281	"DE14"	'postal code'@en	E1
Q1000597	Q3957	P373	"Burton upon Trent"	'Commons category'@en	E2
Q1000597	Q3957	P473	"01283"	'local dialing code'@en	E3
Q1000597	Q3957	P613	"SK245225"	'OS grid reference'@en	E4


Aggregating distinct labels w/ positive entity counts

In [442]:
!kgtk query -i $OUT/$NAME.entity_attribute_labels_string.tsv \
-o $OUT/$NAME.candidate_labels_avl_string_temp.tsv --graph-cache $STORE \
--match 'labels: (type)-[l {label:prop, property_label:lab, entity:e}]->(val)' \
--return 'distinct type as type, prop as prop, val as value, count(distinct e) as positives, lab as property_label, "_" as id' \
--order-by 'count(distinct e) desc'

In [443]:
!kgtk rename-columns -i $OUT/$NAME.candidate_labels_avl_string_temp.tsv \
-o $OUT/$NAME.candidate_labels_avl_string_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [444]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.candidate_labels_avl_string_temp1.tsv \
-o $OUT/$NAME.candidate_labels_avl_string_temp2.tsv

In [445]:
!kgtk add-id -i $OUT/$NAME.candidate_labels_avl_string_temp2.tsv -o $OUT/$NAME.candidate_labels_avl_string.tsv

In [446]:
!head -5 $OUT/$NAME.candidate_labels_avl_string.tsv

node1	label	node2	positives	property_label	id
Q3624078	P3238	"0"	34	'trunk prefix'@en	E1
Q3624078	P3238	novalue	14	'trunk prefix'@en	E2
Q6256	P3238	"0"	12	'trunk prefix'@en	E3
Q179164	P3238	"0"	9	'trunk prefix'@en	E4


### Times

Looking at what precisions we need to deal with...

In [525]:
!kgtk query -i $DATA/$NAME.part.time.tsv $DATA/$NAME.label.en.tsv\
--graph-cache $STORE \
--match 'time: (n1)-[l {label:p}]->(n2), label: (p)-[:label]->(lab)' \
--return 'distinct kgtk_date_precision(n2) as precisions, count(n1) as count' \
--limit 10

precisions	count
6	12
7	40
8	12
9	697
10	48
11	469


From the above, we have several precisions below precision of year=9. We don't have kgtk type interpretation functions for these granularities, so for now we'll interpret them all as years. We also don't have a kgtk function for month and year - just month alone. We'll use the date (with day, month, and year) for precisions finer than a year. **Note** we may want to find a way to eventually recover the original intended type.

#### precision = year or broader
We interpret these as years

In [526]:
!kgtk query -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.part.time.tsv -i $DATA/$NAME.label.en.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.year_temp.tsv --graph-cache $STORE \
--match 'time: (n1)-[l {label:p}]->(n2), type: (n1)-[]->(type), label: (p)-[:label]->(lab)' \
--return 'n1 as entity, type as type, p as prop, kgtk_date_year(n2) as value, lab as property_label, "_" as id' \
--where 'lab.kgtk_lqstring_lang_suffix = "en" AND kgtk_date_precision(n2) <= 9' \
--order-by 'n1'

In [527]:
!kgtk rename-columns -i $OUT/$NAME.entity_attribute_labels_time.year_temp.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.year_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [528]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.entity_attribute_labels_time.year_temp1.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.year_temp2.tsv

In [529]:
!kgtk add-id -i $OUT/$NAME.entity_attribute_labels_time.year_temp2.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.year.tsv

In [530]:
!head -5 $OUT/$NAME.entity_attribute_labels_time.year.tsv

entity	node1	label	node2	property_label	id
Q1019	Q112099	P571	1960	'inception'@en	E1
Q1019	Q3624078	P571	1960	'inception'@en	E2
Q1019	Q112099	P571	1960	'inception'@en	E3
Q1019	Q3624078	P571	1960	'inception'@en	E4


Aggregating distinct labels w/ positive entity counts

In [531]:
!kgtk query -i $OUT/$NAME.entity_attribute_labels_time.year.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.year_temp.tsv --graph-cache $STORE \
--match 'labels: (n1)-[l {label:p, property_label:lab, entity:e}]->(val)' \
--return 'distinct n1 as type, p as prop, val as value, count(distinct e) as positives, lab as property_label, "_" as id' \
--order-by 'count(distinct e) desc'

In [532]:
!kgtk rename-columns -i $OUT/$NAME.candidate_labels_avl_time.year_temp.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.year_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [533]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.candidate_labels_avl_time.year_temp1.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.year_temp2.tsv

In [534]:
!kgtk add-id -i $OUT/$NAME.candidate_labels_avl_time.year_temp2.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.year.tsv

In [535]:
!head -5 $OUT/$NAME.candidate_labels_avl_time.year.tsv

node1	label	node2	positives	property_label	id
Q3624078	P571	1991	3	'inception'@en	E1
Q123480	P571	1991	2	'inception'@en	E2
Q131734	P571	1985	2	'inception'@en	E3
Q131734	P571	1996	2	'inception'@en	E4


#### precision finer than a year
we interpret these as dates with day, month, year

In [536]:
!kgtk query -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.part.time.tsv -i $DATA/$NAME.label.en.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.date_temp.tsv --graph-cache $STORE \
--match 'time: (n1)-[l {label:p}]->(n2), type: (n1)-[]->(type), label: (p)-[:label]->(lab)' \
--return 'n1 as entity, type as type, p as prop, kgtk_date_date(n2) as value, lab as property_label, "_" as id' \
--where 'lab.kgtk_lqstring_lang_suffix = "en" AND kgtk_date_precision(n2) > 9' \
--order-by 'n1'

In [537]:
!kgtk rename-columns -i $OUT/$NAME.entity_attribute_labels_time.date_temp.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.date_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [538]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.entity_attribute_labels_time.date_temp1.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.date_temp2.tsv

In [539]:
!kgtk add-id -i $OUT/$NAME.entity_attribute_labels_time.date_temp2.tsv \
-o $OUT/$NAME.entity_attribute_labels_time.date.tsv

In [540]:
!head -5 $OUT/$NAME.entity_attribute_labels_time.date.tsv

entity	node1	label	node2	property_label	id
Q1011	Q112099	P571	^1975-07-05	'inception'@en	E1
Q1011	Q3624078	P571	^1975-07-05	'inception'@en	E2
Q1011	Q112099	P571	^1975-07-05	'inception'@en	E3
Q1011	Q3624078	P571	^1975-07-05	'inception'@en	E4


Aggregating distinct labels w/ positive entity counts

In [541]:
!kgtk query -i $OUT/$NAME.entity_attribute_labels_time.date.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.date_temp.tsv --graph-cache $STORE \
--match 'labels: (n1)-[l {label:p, property_label:lab, entity:e}]->(val)' \
--return 'distinct n1 as type, p as prop, val as value, count(distinct e) as positives, lab as property_label, "_" as id' \
--order-by 'count(distinct e) desc'

In [542]:
!kgtk rename-columns -i $OUT/$NAME.candidate_labels_avl_time.date_temp.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.date_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [543]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.candidate_labels_avl_time.date_temp1.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.date_temp2.tsv

In [544]:
!kgtk add-id -i $OUT/$NAME.candidate_labels_avl_time.date_temp2.tsv \
-o $OUT/$NAME.candidate_labels_avl_time.date.tsv

In [545]:
!head -5 $OUT/$NAME.candidate_labels_avl_time.date.tsv

node1	label	node2	positives	property_label	id
Q1066984	P1249	^1158-06-14	1	'time of earliest written record'@en	E1
Q1093829	P571	^1790-07-16	1	'inception'@en	E2
Q112099	P571	^-0660-02-11	1	'inception'@en	E3
Q112099	P571	^1898-12-10	1	'inception'@en	E4


### Combining entity --> attribute label mappings to single table

In [548]:
!kgtk cat \
-i $OUT/$NAME.entity_attribute_labels_string.tsv \
-i $OUT/$NAME.entity_attribute_labels_time.year.tsv \
-i $OUT/$NAME.entity_attribute_labels_time.date.tsv \
-o $OUT/$NAME.entity_attribute_labels_all.tsv

## 3. Create RELs with counts of positive entities

In [478]:
!kgtk query -i $DATA/$NAME.part.wikibase-item.tsv -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.label.en.tsv \
-o $OUT/$NAME.candidate_labels_rel_item_temp.tsv --graph-cache $STORE \
--match 'item: (n1)-[l {label:p}]->(n2), type: (n1)-[]->(type), label: (p)-[:label]->(lab)' \
--return 'distinct type as type, p as prop, n2 as value, count(distinct n1) as positives, lab as property_label, "_" as id' \
--where 'lab.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'count(distinct n1) desc'

In [479]:
!kgtk rename-columns -i $OUT/$NAME.candidate_labels_rel_item_temp.tsv \
-o $OUT/$NAME.candidate_labels_rel_item_temp1.tsv \
--old-columns type prop value --new-columns node1 label node2

In [480]:
!kgtk remove-columns -c "id" -i $OUT/$NAME.candidate_labels_rel_item_temp1.tsv \
-o $OUT/$NAME.candidate_labels_rel_item_temp2.tsv

In [481]:
!kgtk add-id -i $OUT/$NAME.candidate_labels_rel_item_temp2.tsv \
-o $OUT/$NAME.candidate_labels_rel_item.tsv

In [482]:
!head -5 $OUT/$NAME.candidate_labels_rel_item.tsv

node1	label	node2	positives	property_label	id
Q131734	P452	Q869095	77	'industry'@en	E1
Q3624078	P463	Q1065	68	'member of'@en	E2
Q3624078	P463	Q7817	67	'member of'@en	E3
Q3624078	P530	Q183	67	'diplomatic relation'@en	E4


## 4. Create AILs with counts of positive entities
### 4.a. Keep track of entities --> matching attribute labels for future use


Just implementing a naive hard-coded bucket-size at first for time.date attribute labels

**issue:** mod operator doesn't seem to be supported here..

**TODO:** make table of type/property --> low_bound/up_bound buckets to be used below instead.

**issue:** to do the above it would be nice to have some -/+ infinity value. Otherwise, need logic like "WHERE low_bound is null OR low_bound < value" - this might work?

In [488]:
!kgtk query -i $OUT/$NAME.entity_attribute_labels_time.year.tsv \
--graph-cache $STORE \
--match 'labels: (n1)-[l {label:p, property_label:lab, entity:e}]->(val)' \
--return 'count(val) as vals' \
--where 'val > 1990 AND val < 2000' \
--limit 10

vals
50


In [489]:
!kgtk query -i $OUT/$NAME.entity_attribute_labels_time.year.tsv \
--graph-cache $STORE \
--match 'labels: (n1)-[l {label:p, property_label:lab, entity:e}]->(val)' \
--return 'e as entity, n1 as type, p as prop, (val - (val % 5)) as low_bound, lab as property_label, "_" as id' \
--order-by 'e' \
--limit 5

Unhandled expression type: ('Mod', {'arg1': ('Variable', {'name': 'val'}), 'arg2': ('Literal', {'value': 5})})



## 5. Create RALs with counts of positive entities

Here is an idea of what kind of values we should find (creating RALs from scratch for string attributes only)

In [502]:
!kgtk query -i $DATA/$NAME.part.wikibase-item.tsv -i $DATA/$NAME.part.string.tsv \
-i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.label.en.tsv --graph-cache $STORE \
--match 'item: (n1)-[l1 {label:p1}]->(n2), type: (n1)-[]->(type1), label: (p1)-[:label]->(lab1), string: (n2)-[l2 {label:p2}]->(n3), type: (n2)-[]->(type2), label: (p2)-[:label]->(lab2)' \
--return 'distinct type1 as type1, p1 as prop1, type2 as type2, p2 as prop2, n3 as value, count(distinct n1) as positives, lab1 as prop1_label, lab2 as prop2_label' \
--where 'lab1.kgtk_lqstring_lang_suffix = "en"' \
--where 'lab2.kgtk_lqstring_lang_suffix = "en"' \
--order-by 'count(distinct n1) desc' \
--limit 5

type1	prop1	type2	prop2	value	positives	prop1_label	prop2_label
Q131734	P452	Q8148	P373	"Beer brewing"	77	'industry'@en	'Commons category'@en
Q3624078	P530	Q3624078	P3238	"0"	68	'diplomatic relation'@en	'trunk prefix'@en
Q3624078	P530	Q4209223	P3238	"0"	68	'diplomatic relation'@en	'trunk prefix'@en
Q3624078	P530	Q43702	P3238	"0"	68	'diplomatic relation'@en	'trunk prefix'@en
Q3624078	P530	Q619610	P3238	"0"	68	'diplomatic relation'@en	'trunk prefix'@en


**NOTE:** It looks like having all of the entity --> labels mappings together in one file would be nice here. I don't think there is a need to keep these separate? Maybe keep AVL and AIL separate.

Now using the REL table and the entities --> attribute labels table that we built in steps 2 and 4. Also keep track of counts of positive entities for each label

Trying to reuse RELs, however, when counting positives, we would need to sum each num_pos that matches the line - not sure how to do this, so the below won't capture when type1 --> x1 and type1 --> x2 resolve to be the same label: i.e. type1 --> typex --> val. See further down for alternate solution.

In [500]:
!kgtk query -i $OUT/$NAME.candidate_labels_rel_item.tsv -i $OUT/$NAME.entity_attribute_labels_string.tsv \
--graph-cache $STORE \
--match 'rel: (t1)-[l1 {label:p1, positives:num_pos}]->(v1), entity_attribute: (t2)-[l {entity:v1, label:p2}]->(v2)' \
--return 't1 as type, p1 as prop, t2 as value_type, p2 as value_prop, v2 as value_val, num_pos as positives, "_" as id' \
--order-by "kgtk_quantity_number_int(num_pos) desc" \
--limit 5

type	prop	value_type	value_prop	value_val	positives	id
Q131734	P452	Q8148	P373	"Beer brewing"	77	_
Q3624078	P530	Q3624078	P2258	"262"	67	_
Q3624078	P530	Q3624078	P2572	"Deutschland"	67	_
Q3624078	P530	Q3624078	P2979	"211"	67	_
Q3624078	P530	Q3624078	P2979	"218"	67	_


This way should work.

In [501]:
!kgtk query -i $DATA/$NAME.part.wikibase-item.tsv -i $OUT/$NAME.type_mapping.tsv \
-i $OUT/$NAME.entity_attribute_labels_string.tsv --graph-cache $STORE \
--match 'item: (n1)-[l1 {label:p1}]->(n2), type: (n1)-[]->(t1), entity_attribute: (t2)-[l2 {label:p2, entity:n2}]->(val)' \
--return 't1 as type1, p1 as prop1, t2 as type2, p2 as prop2, val as value, count(distinct n1) as positives, "_" as id' \
--order-by "count(distinct n1) desc" \
--limit 5

type1	prop1	type2	prop2	value	positives	id
Q131734	P452	Q8148	P373	"Beer brewing"	77	_
Q3624078	P530	Q3624078	P3238	"0"	68	_
Q3624078	P530	Q4209223	P3238	"0"	68	_
Q3624078	P530	Q43702	P3238	"0"	68	_
Q3624078	P530	Q619610	P3238	"0"	68	_


And now doing this for all attribute types:

In [552]:
!kgtk query -i $DATA/$NAME.part.wikibase-item.tsv -i $OUT/$NAME.type_mapping.tsv -i $DATA/$NAME.label.en.tsv \
-i $OUT/$NAME.entity_attribute_labels_all.tsv --graph-cache $STORE \
--match 'item: (n1)-[l1 {label:p1}]->(n2), type: (n1)-[]->(t1), entity_attribute: (t2)-[l2 {label:p2, entity:n2}]->(val), label: (p2)-[:label]->(lab2)' \
--return 't1 as type1, p1 as prop1, t2 as type2, p2 as prop2, lab2 as prop2_label, val as value, count(distinct n1) as positives, "_" as id' \
--order-by "count(distinct n1) desc" \
--where 'lab2.kgtk_lqstring_lang_suffix = "en"' \
--limit 5

type1	prop1	type2	prop2	prop2_label	value	positives	id
Q131734	P452	Q8148	P373	'Commons category'@en	"Beer brewing"	77	_
Q131734	P452	Q8148	P580	'start time'@en	-3500	77	_
Q3624078	P530	Q3624078	P3238	'trunk prefix'@en	"0"	68	_
Q3624078	P530	Q4209223	P3238	'trunk prefix'@en	"0"	68	_
Q3624078	P530	Q43702	P3238	'trunk prefix'@en	"0"	68	_


## 6. Filtering candidate labels - This can go in a separate notebook
Simple rule-based filter to remove labels that are trivially either unrepresentative or indistinctive